In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from scipy.stats import norm ,skew,kurtosis
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import make_scorer,mean_squared_error

In [2]:
df_train=pd.read_csv('F:\\study\\ml\\DataSet\\Tap4fun\\tap_fun_train.csv')

In [3]:
c_int=df_train.select_dtypes(include=['int64']).apply(pd.to_numeric,downcast='unsigned')
c_float=df_train.select_dtypes(include=['float']).apply(pd.to_numeric,downcast='float')

In [4]:
o_df=pd.concat([c_int,c_float],axis=1)
column_type=dict(o_df.dtypes.copy())
del o_df,c_int,c_float,df_train

In [5]:
data_train=pd.read_csv('F:\\study\\ml\\DataSet\\Tap4fun\\tap_fun_train.csv',
                      dtype=column_type,parse_dates=['register_time'],infer_datetime_format=True)

data_test=pd.read_csv('F:\\study\\ml\\DataSet\\Tap4fun\\tap_fun_test.csv',
                      dtype=column_type,parse_dates=['register_time'],infer_datetime_format=True)

In [6]:
data_train.rename(columns={'treatment_acceleraion_add_value':'treatment_acceleration_add_value',\
                          'sr_rss_a_prod_levell':'sr_rss_a_prod_level'} ,inplace=True)

data_test.rename(columns={'treatment_acceleraion_add_value':'treatment_acceleration_add_value',\
                          'sr_rss_a_prod_levell':'sr_rss_a_prod_level'} ,inplace=True)

In [7]:
train_user=data_train.iloc[:,:2]
train_data=data_train.iloc[:,2:]

In [8]:
column_name=train_data.columns.values[:32].tolist()
for i in range(len(column_name)):
    if (i+2) % 2 == 0 :
        name=column_name[i].replace('value','rate')
        #print(name)
        train_data[name]=train_data[column_name[i]] / (train_data[column_name[i+1]]+1)

In [9]:
train_data['pvp_lanch_battle_rate']=train_data['pvp_battle_count']/(train_data['pvp_lanch_count']+1)
train_data['pvp_win_battle_rate']=train_data['pvp_battle_count']/(train_data['pvp_win_count']+1)
train_data['pvp_win_lanch_rate']=train_data['pvp_lanch_count']/(train_data['pvp_win_count']+1)
train_data['pve_win_battle_rate']=train_data['pvp_battle_count']/(train_data['pve_win_count']+1)
train_data['pay_price_rate']=train_data['pay_count']/(train_data['pay_price']+1)
train_data['price_online_minutes_rate']=train_data['avg_online_minutes']/(train_data['pay_price']+1)

In [10]:
train_data['resource_add_mean']=train_data.iloc[:,[0,2,4,6,8]].apply(lambda x:x.mean(),axis=1)
train_data['resource_add_med']=train_data.iloc[:,[0,2,4,6,8]].apply(lambda x:x.median(),axis=1)
train_data['resource_reduce_mean']=train_data.iloc[:,[1, 3, 5, 7, 9]].apply(lambda x:x.mean(),axis=1)
train_data['resource_reduce_med']=train_data.iloc[:,[1, 3, 5, 7, 9]].apply(lambda x:x.median(),axis=1)

train_data['resource_mean_rate']=train_data['resource_reduce_mean']/(train_data['resource_add_mean']+1)
train_data['resource_med_rate']=train_data['resource_reduce_med']/(train_data['resource_add_med']+1)

train_data['army_add_mean']=train_data.iloc[:,[10, 12, 14]].apply(lambda x:x.mean(),axis=1)
train_data['army_add_med']=train_data.iloc[:,[10, 12, 14]].apply(lambda x:x.median(),axis=1)
train_data['army_reduce_mean']=train_data.iloc[:,[11, 13, 15]].apply(lambda x:x.mean(),axis=1)
train_data['army_reduce_med']=train_data.iloc[:,[11, 13, 15]].apply(lambda x:x.median(),axis=1)

train_data['army_mean_rate']=train_data['army_reduce_mean']/(train_data['army_add_mean']+1)
train_data['army_med_rate']=train_data['army_reduce_med']/(train_data['army_add_med']+1)

train_data['wound_add_mean'] = train_data.iloc[:, [16, 18, 20]].apply(lambda x: x.mean(), axis=1)
train_data['wound_add_med'] = train_data.iloc[:, [16, 18, 20]].apply(lambda x: x.median(), axis=1)
train_data['wound_reduce_mean'] = train_data.iloc[:, [17, 19, 21]].apply(lambda x: x.mean(), axis=1)
train_data['wound_reduce_med'] = train_data.iloc[:, [17, 19, 21]].apply(lambda x: x.median(), axis=1)

train_data['wound_mean_rate']=train_data['wound_reduce_mean']/(train_data['wound_add_mean']+1)
train_data['wound_med_rate']=train_data['wound_reduce_med']/(train_data['wound_add_med']+1)

train_data['accelerate_add_mean'] = train_data.iloc[:, [22, 24, 26, 28, 30]].apply(lambda x: x.mean(), axis=1)
train_data['accelerate_add_med'] = train_data.iloc[:, [22, 24, 26, 28, 30]].apply(lambda x: x.median(), axis=1)
train_data['accelerate_reduce_mean'] = train_data.iloc[:, [23, 25, 27, 29, 31]].apply(lambda x: x.mean(), axis=1)
train_data['accelerate_reduce_med'] = train_data.iloc[:, [23, 25, 27, 29, 31]].apply(lambda x: x.median(), axis=1)

train_data['accelerate_mean_rate']=train_data['accelerate_reduce_mean']/(train_data['accelerate_add_mean']+1)
train_data['accelerate_med_rate']=train_data['accelerate_reduce_med']/(train_data['accelerate_add_med']+1)

In [11]:
train_user['register_time_month']=pd.to_numeric(train_user.register_time.apply(lambda x:str(x)).str[5:7])
train_user['register_time_day']=pd.to_numeric(train_user.register_time.apply(lambda x:str(x)).str[8:10])
train_user.drop(labels=['register_time'],axis=1,inplace=True)
train_user['register_time_count']=train_user['register_time_month']*31+train_user['register_time_day']

train_data['register_time_month']=train_user['register_time_month']
train_data['register_time_day']=train_user['register_time_day']
train_data['register_time_count']=train_user['register_time_count']

In [12]:
train = train_data[train_data['pay_price'] > 0]

In [13]:
train['new_pay_label']=train['prediction_pay_price']!=train['pay_price']
train['new_pay_label']=train['new_pay_label'].map({True:1,False:0})
train['new_pay_price']=train['prediction_pay_price']-train['pay_price']
train['new_pay_rate']=(train['prediction_pay_price']-train['pay_price'])/train['pay_price']

D:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
D:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
D:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.

In [14]:
train.head()

,wood_add_value,wood_reduce_value,stone_add_value,stone_reduce_value,ivory_add_value,ivory_reduce_value,meat_add_value,meat_reduce_value,magic_add_value,magic_reduce_value,...,accelerate_reduce_mean,accelerate_reduce_med,accelerate_mean_rate,accelerate_med_rate,register_time_month,register_time_day,register_time_count,new_pay_label,new_pay_price,new_pay_rate
25,30000.0,97200.0,20000.0,0.0,0.0,0.0,160500.0,80392.0,12000.0,0.0,...,0.0,0.0,0.000000,0.000000,1,26,57,0,0.00,0.000000
40,1111744.0,1137687.0,491331.0,790208.0,77000.0,0.0,1457249.0,708483.0,217455.0,162987.0,...,322.8,500.0,0.510598,0.807754,1,26,57,1,1.99,0.670034
47,249050.0,173248.0,70841.0,117012.0,55000.0,0.0,206366.0,82205.0,29902.0,88064.0,...,415.8,0.0,1.825285,0.000000,1,26,57,0,0.00,0.000000
86,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.000000,1,26,57,0,0.00,0.000000
119,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.000000,1,26,57,0,0.00,0.000000


In [15]:
train.iloc[:,0:-3].head()

,wood_add_value,wood_reduce_value,stone_add_value,stone_reduce_value,ivory_add_value,ivory_reduce_value,meat_add_value,meat_reduce_value,magic_add_value,magic_reduce_value,...,wound_med_rate,accelerate_add_mean,accelerate_add_med,accelerate_reduce_mean,accelerate_reduce_med,accelerate_mean_rate,accelerate_med_rate,register_time_month,register_time_day,register_time_count
25,30000.0,97200.0,20000.0,0.0,0.0,0.0,160500.0,80392.0,12000.0,0.0,...,0.0,100.0,75.0,0.0,0.0,0.000000,0.000000,1,26,57
40,1111744.0,1137687.0,491331.0,790208.0,77000.0,0.0,1457249.0,708483.0,217455.0,162987.0,...,0.0,631.2,618.0,322.8,500.0,0.510598,0.807754,1,26,57
47,249050.0,173248.0,70841.0,117012.0,55000.0,0.0,206366.0,82205.0,29902.0,88064.0,...,0.0,226.8,45.0,415.8,0.0,1.825285,0.000000,1,26,57
86,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,1,26,57
119,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,1,26,57


In [16]:
train.head()

,wood_add_value,wood_reduce_value,stone_add_value,stone_reduce_value,ivory_add_value,ivory_reduce_value,meat_add_value,meat_reduce_value,magic_add_value,magic_reduce_value,...,accelerate_reduce_mean,accelerate_reduce_med,accelerate_mean_rate,accelerate_med_rate,register_time_month,register_time_day,register_time_count,new_pay_label,new_pay_price,new_pay_rate
25,30000.0,97200.0,20000.0,0.0,0.0,0.0,160500.0,80392.0,12000.0,0.0,...,0.0,0.0,0.000000,0.000000,1,26,57,0,0.00,0.000000
40,1111744.0,1137687.0,491331.0,790208.0,77000.0,0.0,1457249.0,708483.0,217455.0,162987.0,...,322.8,500.0,0.510598,0.807754,1,26,57,1,1.99,0.670034
47,249050.0,173248.0,70841.0,117012.0,55000.0,0.0,206366.0,82205.0,29902.0,88064.0,...,415.8,0.0,1.825285,0.000000,1,26,57,0,0.00,0.000000
86,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.000000,1,26,57,0,0.00,0.000000
119,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.000000,1,26,57,0,0.00,0.000000


In [17]:
classify_train=train.iloc[:,0:-3]
classify_train.drop('prediction_pay_price',axis=1,inplace=True)

In [18]:
classify_train.head()

,wood_add_value,wood_reduce_value,stone_add_value,stone_reduce_value,ivory_add_value,ivory_reduce_value,meat_add_value,meat_reduce_value,magic_add_value,magic_reduce_value,...,wound_med_rate,accelerate_add_mean,accelerate_add_med,accelerate_reduce_mean,accelerate_reduce_med,accelerate_mean_rate,accelerate_med_rate,register_time_month,register_time_day,register_time_count
25,30000.0,97200.0,20000.0,0.0,0.0,0.0,160500.0,80392.0,12000.0,0.0,...,0.0,100.0,75.0,0.0,0.0,0.000000,0.000000,1,26,57
40,1111744.0,1137687.0,491331.0,790208.0,77000.0,0.0,1457249.0,708483.0,217455.0,162987.0,...,0.0,631.2,618.0,322.8,500.0,0.510598,0.807754,1,26,57
47,249050.0,173248.0,70841.0,117012.0,55000.0,0.0,206366.0,82205.0,29902.0,88064.0,...,0.0,226.8,45.0,415.8,0.0,1.825285,0.000000,1,26,57
86,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,1,26,57
119,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,1,26,57


In [19]:
train.shape

(41439, 159)

In [20]:
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier
from sklearn.externals import joblib
from sklearn.metrics import classification_report,accuracy_score,confusion_matrix
from sklearn.cross_validation import train_test_split,cross_val_score
import lightgbm  as lgb

D:\ProgramData\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [21]:
# classify_train=train.iloc[:,1:-4]
cl_t = classify_train.iloc[:, :-3].drop_duplicates()
X_train,X_test,y_train,y_test=train_test_split(cl_t,train.loc[cl_t.index,'new_pay_label'],test_size=0.2,random_state=13)


In [22]:
model44= GridSearchCV(
    estimator=lgb.LGBMClassifier(),
    param_grid={
        'num_leaves':[114],
        'n_estimators': [1000],
        'learning_rate': [0.064],
        'max_depth': [3],
        'subsample': [1],
         'subsample': [0.8],
        'colsample_bytree': [0.56],
        'scale_pos_weight': [2.5, ],
#         'min_child_weight': [2, ]
    },
    scoring='f1',
    cv=3,
    n_jobs=4,
    verbose=1).fit(X_train,y_train)

ypred44=model44.predict(X_test)
predictions=ypred44
print('准确率',accuracy_score(y_test,predictions))
print('混淆矩阵',confusion_matrix(y_test,predictions))
print('分类报告',classification_report(y_test,predictions))

Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=4)]: Done   3 out of   3 | elapsed:   12.2s finished


准确率 0.7771931971124434
混淆矩阵 [[4576 1302]
 [ 519 1776]]
分类报告              precision    recall  f1-score   support

          0       0.90      0.78      0.83      5878
          1       0.58      0.77      0.66      2295

avg / total       0.81      0.78      0.79      8173



D:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [23]:
from xgboost import XGBRegressor

In [24]:
lgbm2=lgb.LGBMClassifier(boosting_type='gbdt',
nthread=4, 
silent=True,
n_estimators= 1000,
learning_rate= 0.064,
max_depth= 3,
subsample= 0.8,
colsample_bytree= 0.56,
scale_pos_weight=2.5).fit(X_train,y_train)

In [25]:
df_fi=pd.DataFrame()
# df_fi['name']=lgbm2.feature_name()
df_fi['importance']=lgbm2.feature_importances_
df_fi.sort_values(by=['importance'],axis=0,ascending=False).head(10)

,importance
103,204
118,157
117,130
119,125
127,120
30,120
26,115
104,111
110,110
28,106


In [26]:
model_reg = GridSearchCV(
    estimator=XGBRegressor(),
    #     estimator=LinearRegression(),
    param_grid={
        'n_estimators': [80],
        'learning_rate': [0.05],
        'max_depth': [2],
        'subsample': [1],
        'colsample_bytree': [0.5],
        'reg_alpha': [13.4],
    },
    scoring='neg_mean_squared_error',
    cv=3,
    n_jobs=1,
    verbose=4)
model_reg.fit(classify_train.iloc[:, [103, 108, 117, 119, 127,30,26,104,110,28]][train['new_pay_label'] == 1],
              train.iloc[:, -1][train['new_pay_label'] == 1])
# model_reg.fit(cl_t.iloc[:,[105,126,121,132,150]][train.loc[cl_t.index,'new_pay_label']==1],train.iloc[cl_t.index,-1][train.loc[cl_t.index,'new_pay_label']==1])
print(np.sqrt(-model_reg.best_score_), model_reg.best_params_)

Fitting 3 folds for each of 1 candidates, totalling 3 fits
[CV] colsample_bytree=0.5, learning_rate=0.05, max_depth=2, n_estimators=80, reg_alpha=13.4, subsample=1 
[CV]  colsample_bytree=0.5, learning_rate=0.05, max_depth=2, n_estimators=80, reg_alpha=13.4, subsample=1, score=-2039.189208984375, total=   0.3s
[CV] colsample_bytree=0.5, learning_rate=0.05, max_depth=2, n_estimators=80, reg_alpha=13.4, subsample=1 
[CV]  colsample_bytree=0.5, learning_rate=0.05, max_depth=2, n_estimators=80, reg_alpha=13.4, subsample=1, score=-1939.345703125, total=   0.1s
[CV] colsample_bytree=0.5, learning_rate=0.05, max_depth=2, n_estimators=80, reg_alpha=13.4, subsample=1 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.5s remaining:    0.0s


[CV]  colsample_bytree=0.5, learning_rate=0.05, max_depth=2, n_estimators=80, reg_alpha=13.4, subsample=1, score=-2777.177978515625, total=   0.1s


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.7s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.7s finished


47.4537443145628 {'colsample_bytree': 0.5, 'learning_rate': 0.05, 'max_depth': 2, 'n_estimators': 80, 'reg_alpha': 13.4, 'subsample': 1}


In [31]:
model_reg2 = GridSearchCV(
    estimator=lgb.LGBMRegressor( max_bin=128),
    #     estimator=LinearRegression(),
    param_grid={
        'n_estimators': [80],
        'learning_rate': [0.05],
        'max_depth': [2],
        'subsample': [0.8],
        'colsample_bytree': [0.56],
#         'reg_alpha': [13.4],
    },
    scoring='neg_mean_squared_error',
    cv=3,
    n_jobs=1,
    verbose=4)
model_reg2.fit(classify_train.iloc[:, [103, 108, 117, 119, 127,30,26,104,110,28]][train['new_pay_label'] == 1],
              train.iloc[:, -1][train['new_pay_label'] == 1])
# model_reg.fit(cl_t.iloc[:,[105,126,121,132,150]][train.loc[cl_t.index,'new_pay_label']==1],train.iloc[cl_t.index,-1][train.loc[cl_t.index,'new_pay_label']==1])
print(np.sqrt(-model_reg2.best_score_), model_reg2.best_params_)

Fitting 3 folds for each of 1 candidates, totalling 3 fits
[CV] colsample_bytree=0.56, learning_rate=0.05, max_depth=2, n_estimators=80, subsample=0.8 
[CV]  colsample_bytree=0.56, learning_rate=0.05, max_depth=2, n_estimators=80, subsample=0.8, score=-2048.98916197549, total=   0.0s
[CV] colsample_bytree=0.56, learning_rate=0.05, max_depth=2, n_estimators=80, subsample=0.8 
[CV]  colsample_bytree=0.56, learning_rate=0.05, max_depth=2, n_estimators=80, subsample=0.8, score=-1937.4693483972294, total=   0.0s
[CV] colsample_bytree=0.56, learning_rate=0.05, max_depth=2, n_estimators=80, subsample=0.8 
[CV]  colsample_bytree=0.56, learning_rate=0.05, max_depth=2, n_estimators=80, subsample=0.8, score=-2781.3616382406467, total=   0.0s
47.49624815768069 {'colsample_bytree': 0.56, 'learning_rate': 0.05, 'max_depth': 2, 'n_estimators': 80, 'subsample': 0.8}


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.1s finished


In [32]:
X_train.iloc[:, [103, 108, 117, 119, 127,30,26,104,110,28]][train['new_pay_label'] == 1].shape

D:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


(9009, 10)

In [33]:
X_train2,X_test2,y_train2,y_test2=train_test_split(cl_t,train.loc[cl_t.index,'prediction_pay_price'],test_size=0.2,random_state=13)

In [34]:
y_train2[train['new_pay_label'] == 1].shape

(9009,)

In [35]:
################# for test data

In [36]:
data_test.head()

,user_id,register_time,wood_add_value,wood_reduce_value,stone_add_value,stone_reduce_value,ivory_add_value,ivory_reduce_value,meat_add_value,meat_reduce_value,...,sr_rss_help_bonus_level,pvp_battle_count,pvp_lanch_count,pvp_win_count,pve_battle_count,pve_lanch_count,pve_win_count,avg_online_minutes,pay_price,pay_count
0,14933,2018-03-08 20:27:57,166415.0,138362.0,0.0,0.0,0.0,0.0,258522.0,90142.0,...,0,0,0,0,1,1,1,8.000000,0.0,0
1,14934,2018-03-08 20:29:42,10000.0,600.0,0.0,0.0,0.0,0.0,10000.0,400.0,...,0,0,0,0,0,0,0,0.166667,0.0,0
2,14935,2018-03-08 20:30:13,10000.0,3700.0,0.0,0.0,0.0,0.0,10000.0,2000.0,...,0,0,0,0,0,0,0,17.000000,0.0,0
3,14936,2018-03-08 20:31:41,210000.0,0.0,200000.0,0.0,200000.0,0.0,610000.0,0.0,...,0,0,0,0,0,0,0,1.666667,0.0,0
4,14937,2018-03-08 20:32:07,11500.0,3700.0,0.0,0.0,0.0,0.0,11000.0,2000.0,...,0,0,0,0,0,0,0,0.333333,0.0,0


In [48]:
test_user=data_test.iloc[:,:2]
test_data=data_test.iloc[:,2:]

In [49]:
test_data.shape

(828934, 106)

In [50]:
column_name2=test_data.columns.values[:32].tolist()
for i in range(len(column_name2)):
    if (i+2) % 2 == 0 :
        name=column_name2[i].replace('value','rate')
        #print(name)
        test_data[name]=test_data[column_name2[i]] / (test_data[column_name2[i+1]]+1)

In [51]:
test_data.shape

(828934, 122)

In [52]:
test_data['pvp_lanch_battle_rate']=test_data['pvp_battle_count']/(test_data['pvp_lanch_count']+1)
test_data['pvp_win_battle_rate']=test_data['pvp_battle_count']/(test_data['pvp_win_count']+1)
test_data['pvp_win_lanch_rate']=test_data['pvp_lanch_count']/(test_data['pvp_win_count']+1)
test_data['pve_win_battle_rate']=test_data['pvp_battle_count']/(test_data['pve_win_count']+1)
test_data['pay_price_rate']=test_data['pay_count']/(test_data['pay_price']+1)
test_data['price_online_minutes_rate']=test_data['avg_online_minutes']/(test_data['pay_price']+1)


test_data['resource_add_mean']=test_data.iloc[:,[0,2,4,6,8]].apply(lambda x:x.mean(),axis=1)
test_data['resource_add_med']=test_data.iloc[:,[0,2,4,6,8]].apply(lambda x:x.median(),axis=1)
test_data['resource_reduce_mean']=test_data.iloc[:,[1, 3, 5, 7, 9]].apply(lambda x:x.mean(),axis=1)
test_data['resource_reduce_med']=test_data.iloc[:,[1, 3, 5, 7, 9]].apply(lambda x:x.median(),axis=1)

test_data['resource_mean_rate']=test_data['resource_reduce_mean']/(test_data['resource_add_mean']+1)
test_data['resource_med_rate']=test_data['resource_reduce_med']/(test_data['resource_add_med']+1)

test_data['army_add_mean']=test_data.iloc[:,[10, 12, 14]].apply(lambda x:x.mean(),axis=1)
test_data['army_add_med']=test_data.iloc[:,[10, 12, 14]].apply(lambda x:x.median(),axis=1)
test_data['army_reduce_mean']=test_data.iloc[:,[11, 13, 15]].apply(lambda x:x.mean(),axis=1)
test_data['army_reduce_med']=test_data.iloc[:,[11, 13, 15]].apply(lambda x:x.median(),axis=1)

test_data['army_mean_rate']=test_data['army_reduce_mean']/(test_data['army_add_mean']+1)
test_data['army_med_rate']=test_data['army_reduce_med']/(test_data['army_add_med']+1)

test_data['wound_add_mean'] = test_data.iloc[:, [16, 18, 20]].apply(lambda x: x.mean(), axis=1)
test_data['wound_add_med'] = test_data.iloc[:, [16, 18, 20]].apply(lambda x: x.median(), axis=1)
test_data['wound_reduce_mean'] = test_data.iloc[:, [17, 19, 21]].apply(lambda x: x.mean(), axis=1)
test_data['wound_reduce_med'] = test_data.iloc[:, [17, 19, 21]].apply(lambda x: x.median(), axis=1)

test_data['wound_mean_rate']=test_data['wound_reduce_mean']/(test_data['wound_add_mean']+1)
test_data['wound_med_rate']=test_data['wound_reduce_med']/(test_data['wound_add_med']+1)

test_data['accelerate_add_mean'] = test_data.iloc[:, [22, 24, 26, 28, 30]].apply(lambda x: x.mean(), axis=1)
test_data['accelerate_add_med'] = test_data.iloc[:, [22, 24, 26, 28, 30]].apply(lambda x: x.median(), axis=1)
test_data['accelerate_reduce_mean'] = test_data.iloc[:, [23, 25, 27, 29, 31]].apply(lambda x: x.mean(), axis=1)
test_data['accelerate_reduce_med'] = test_data.iloc[:, [23, 25, 27, 29, 31]].apply(lambda x: x.median(), axis=1)

test_data['accelerate_mean_rate']=test_data['accelerate_reduce_mean']/(test_data['accelerate_add_mean']+1)
test_data['accelerate_med_rate']=test_data['accelerate_reduce_med']/(test_data['accelerate_add_med']+1)

# classify_test=test.iloc[:,0:-3]

In [38]:
test_data.shape

(828934, 136)

In [58]:
test_user['register_time_month']=pd.to_numeric(test_user.register_time.apply(lambda x:str(x)).str[5:7])
test_user['register_time_day']=pd.to_numeric(test_user.register_time.apply(lambda x:str(x)).str[8:10])
test_user.drop(labels=['register_time'],axis=1,inplace=True)
test_user['register_time_count']=test_user['register_time_month']*31+test_user['register_time_day']
test_data['register_time_month']=test_user['register_time_month']
test_data['register_time_day']=test_user['register_time_day']
test_data['register_time_count']=test_user['register_time_count']
test = test_data[test_data['pay_price'] > 0]

In [59]:
test_data.shape

(828934, 155)

In [60]:
classify_test=test.iloc[:,:-3]

In [61]:
classify_test.shape

(19549, 152)

In [62]:
test_pred=model44.predict(classify_test)
classify_test2=classify_test
classify_test2['new_pay_label']=test_pred

D:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [63]:
classify_test2.shape

(19549, 153)

In [64]:
classify_test2[classify_test2['new_pay_label']==1].iloc[:, [103, 108, 117, 119, 127,30,26,104,110,28]].shape

(7131, 10)

In [65]:
test_reg_pred=model_reg2.predict(classify_test2[classify_test2['new_pay_label']==1].iloc[:, [103, 108, 117, 119, 127,30,26,104,110,28]])

In [66]:
result1=classify_test2

In [67]:
result1.loc[result1['new_pay_label']==1,'prediction_pay_price']=test_reg_pred

In [68]:
result1[result1['new_pay_label']==1][['pay_price','prediction_pay_price']].head()

,pay_price,prediction_pay_price
470,93.900002,3.440450
558,9.940000,7.700574
559,16.959999,4.305765
768,5.950000,10.174584
908,26.930000,3.903112


In [69]:
result1['prediction_pay_price2']=result1['pay_price'] *  result1['prediction_pay_price']

In [70]:
 result1.loc[result1['new_pay_label'] == 0,'prediction_pay_price2']=result1['pay_price']

In [71]:
test_user['prediction_pay_price']=0.0

In [72]:
test_user.iloc[(result1['prediction_pay_price2']>0).index,4]=result1['prediction_pay_price2']

In [73]:
test_result=pd.DataFrame()
test_result['user_id']=test_user.user_id
test_result['prediction_pay_price']=test_user.prediction_pay_price

In [74]:
result=test_result
import datetime
nowTime=datetime.datetime.now().strftime('%Y%m%d%H%M%S')
name='tf_'+str(nowTime)
result.to_csv("F:\\study\\ml\\DataSet\\Tap4fun\\SubMission\\"+name+'.csv',index=False,encoding='utf-8')
